In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 275 ms (started: 2021-09-05 13:43:17 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #3 Sept 05, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_study.git (fetch)
origin	git@github.com:ustchope/pytorch_study.git (push)
[master 0c11324] 更新 #3 Sept 05, 2021
 8 files changed, 9469 insertions(+), 4 deletions(-)
 create mode 100644 "Pytorch_recipes/.ipynb_checkpoints/\344\273\200\344\271\210\346\230\257 PYTORCH \344\270\255\347\232\204 STATE_DICT-checkpoint.ipynb"
 create mode 100644 "Pytorch_recipes/.ipynb_checkpoints/\345\234\250 PYTORCH \344\270\255\344\277\235\345\255\230\345\222\214\345\212\240\350\275\275\346\216\250\347\220\206\346\250\241\345\236\213-checkpoint.ipynb"
 create mode 100644 "Pytorch_recipes/.ipynb_checkpoints/\345\234\250 PYTORCH \344\270\255\345\256\232\344\271\211\347\245\236\347\273\217\347\275\221\347\273\234-checkpoint.ipynb"
 create mode 100644 "Pytorch_recipes/\344\273\200\344\271\210\346\230\257 PYTORCH \344\270\255\347\232\204 STATE_DICT.ipynb"
 create mode 100644 "Pytorch_recipes/\345\234\250 PYTORCH \344\270\255\344\277\235\345\255\230\345\222\214\345\212\240\35

To github.com:ustchope/pytorch_study.git
   aff92ce..0c11324  master -> master


time: 4.69 s (started: 2021-09-05 13:43:23 +08:00)


在 PyTorch 中有两种保存和加载模型以进行推理的方法。第一个是保存和加载 state_dict，第二个是保存和加载整个模型。

# 介绍

使用 torch.save() 函数保存模型的 state_dict 将为您以后恢复模型提供最大的灵活性。这是保存模型的推荐方法，因为只有真正需要保存训练模型的学习参数。保存和加载整个模型时，您可以使用 Python 的 pickle 模块保存整个模块。使用这种方法会产生最直观的语法并涉及最少的代码。这种方法的缺点是序列化的数据绑定到特定的类和保存模型时使用的确切目录结构。这样做的原因是 pickle 不保存模型类本身。相反，它保存包含类的文件的路径，该路径在加载时使用。因此，您的代码在其他项目中使用或在重构后可能会以各种方式中断。在这个秘籍中，我们将探讨如何保存和加载模型以进行推理的两种方法。

# 步骤
* 导入所有必要的库以加载我们的数据
* 定义并初始化神经网络
* 初始化优化器
* 通过 state_dict 保存和加载模型
* 保存并加载整个模型

1. 导入必要的库来加载我们的数据

对于这个秘籍，我们将使用 torch 及其子公司 torch.nn 和 torch.optim。

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

time: 704 ms (started: 2021-09-05 13:44:26 +08:00)


2. 定义并初始化神经网络

例如，我们将创建一个用于训练图像的神经网络。 要了解更多信息，请参阅定义神经网络食谱。

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
time: 28.6 ms (started: 2021-09-05 13:44:55 +08:00)


3。 初始化优化器

我们将使用动量的 SGD。

In [5]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

time: 1.08 ms (started: 2021-09-05 13:45:18 +08:00)


4. 通过 state_dict 保存和加载模型

让我们只使用 state_dict 来保存和加载我们的模型。

In [6]:
# 指定路径
PATH = "state_dict_model.pt"

# Save
torch.save(net.state_dict(), PATH)

# Load
model = Net()
model.load_state_dict(torch.load(PATH))
model.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

time: 14.5 ms (started: 2021-09-05 13:45:48 +08:00)


一个常见的 PyTorch 约定是使用 .pt 或 .pth 文件扩展名保存模型。

请注意， load_state_dict() 函数采用字典对象，而不是保存对象的路径。 这意味着在将保存的 state_dict 传递给 load_state_dict() 函数之前，您必须对其进行反序列化。 例如，您不能使用 model.load_state_dict(PATH) 加载。

还要记住，在运行推理之前，您必须调用 model.eval() 将 dropout 和批量归一化层设置为评估模式。 不这样做会产生不一致的推理结果。

5. 保存和加载整个模型

现在让我们对整个模型尝试同样的事情。

In [7]:
# Specify a path
PATH = "entire_model.pt"

# Save
torch.save(net, PATH)

# Load
model = torch.load(PATH)
model.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

time: 8.58 ms (started: 2021-09-05 13:47:06 +08:00)


同样，请记住，在运行推理之前，您必须调用 model.eval() 将 dropout 和批量归一化层设置为评估模式。

恭喜！ 您已成功保存和加载模型以在 PyTorch 中进行推理。